In [1]:
def readGenome(filename):
    genome = ''
    with open(r"C:\Users\liamm\Desktop\chr1.GRCh38.excerpt.fasta", 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome


In [2]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [3]:
def naive(p, t):
    occurrences = []
    align_count = 0
    compare_count = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):# loop over characters
            compare_count += 1
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
        align_count += 1
    return occurrences,align_count,compare_count

In [4]:
s = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'

In [5]:
genome = readGenome(r"C:\Users\liamm\Desktop\chr1.GRCh38.excerpt.fasta")

In [6]:
genome[:100]

'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAA'

In [7]:
naive(s,genome)

([56922], 799954, 984143)

In [8]:
import string

def z_array(s):
    """ Use Z algorithm (Gusfield theorem 1.4.1) to preprocess s """
    assert len(s) > 1
    z = [len(s)] + [0] * (len(s)-1)

    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            z[1] += 1
        else:
            break
    
    r, l = 0, 0
    if z[1] > 0:
        r, l = z[1], 1
    
    for k in range(2, len(s)):
        assert z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    z[k] += 1
                else:
                    break
            r, l = k + z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            zkp = z[k - l]
            if nbeta > zkp:
                # Case 2a: Zkp wins
                z[k] = zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                z[k] = r - k + 1
    return z


def n_array(s):
    """ Compile the N array (Gusfield theorem 2.2.2) from the Z array """
    return z_array(s[::-1])[::-1]


def big_l_prime_array(p, n):
    """ Compile L' array (Gusfield theorem 2.2.2) using p and N array.
        L'[i] = largest index j less than n such that N[j] = |P[i:]| """
    lp = [0] * len(p)
    for j in range(len(p)-1):
        i = len(p) - n[j]
        if i < len(p):
            lp[i] = j + 1

    return lp


def big_l_array(p, lp):
    """ Compile L array (Gusfield theorem 2.2.2) using p and L' array.
        L[i] = largest index j less than n such that N[j] >= |P[i:]| """
    l = [0] * len(p)
    l[1] = lp[1]
    for i in range(2, len(p)):
        l[i] = max(l[i-1], lp[i])
    return l


def small_l_prime_array(n):
    """ Compile lp' array (Gusfield theorem 2.2.4) using N array. """
    small_lp = [0] * len(n)
    for i in range(len(n)):
        if n[i] == i+1:  # prefix matching a suffix
            small_lp[len(n)-i-1] = i+1
    for i in range(len(n)-2, -1, -1):  # "smear" them out to the left
        if small_lp[i] == 0:
            small_lp[i] = small_lp[i+1]
    return small_lp


def good_suffix_table(p):
    """ Return tables needed to apply good suffix rule. """
    n = n_array(p)
    lp = big_l_prime_array(p, n)
    return lp, big_l_array(p, lp), small_l_prime_array(n)


def good_suffix_mismatch(i, big_l_prime, small_l_prime):
    """ Given a mismatch at offset i, and given L/L' and l' arrays,
        return amount to shift as determined by good suffix rule. """
    length = len(big_l_prime)
    assert i < length
    if i == length - 1:
        return 0
    i += 1  # i points to leftmost matching position of P
    if big_l_prime[i] > 0:
        return length - big_l_prime[i]
    return length - small_l_prime[i]


def good_suffix_match(small_l_prime):
    """ Given a full match of P to T, return amount to shift as
        determined by good suffix rule. """
    return len(small_l_prime) - small_l_prime[1]


def dense_bad_char_tab(p, amap):
    """ Given pattern string and list with ordered alphabet characters, create
        and return a dense bad character table.  Table is indexed by offset
        then by character. """
    tab = []
    nxt = [0] * len(amap)
    for i in range(0, len(p)):
        c = p[i]
        assert c in amap
        tab.append(nxt[:])
        nxt[amap[c]] = i+1
    return tab


class BoyerMoore(object):
    """ Encapsulates pattern and associated Boyer-Moore preprocessing. """
    
    def __init__(self, p, alphabet='ACGT'):
        self.p = p
        self.alphabet = alphabet

        # Create map from alphabet characters to integers
        self.amap = {}
        for i in range(len(self.alphabet)):
            self.amap[self.alphabet[i]] = i

        # Make bad character rule table
        self.bad_char = dense_bad_char_tab(p, self.amap)

        # Create good suffix rule table
        _, self.big_l, self.small_l_prime = good_suffix_table(p)

    def bad_character_rule(self, i, c):
        """ Return # skips given by bad character rule at offset i """
        assert c in self.amap
        ci = self.amap[c]
        assert i > (self.bad_char[i][ci]-1)
        return i - (self.bad_char[i][ci]-1)

    def good_suffix_rule(self, i):
        """ Given a mismatch at offset i, return amount to shift
            as determined by (weak) good suffix rule. """
        length = len(self.big_l)
        assert i < length
        if i == length - 1:
            return 0
        i += 1  # i points to leftmost matching position of P
        if self.big_l[i] > 0:
            return length - self.big_l[i]
        return length - self.small_l_prime[i]

    def match_skip(self):
        """ Return amount to shift in case where P matches T """
        return len(self.small_l_prime) - self.small_l_prime[1]

In [9]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching """
    i = 0
    occurrences = []
    align_count = 0 #number of alignment when matching the string with the reference genome
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        align_count += 1
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences,align_count

In [10]:
s_bm = BoyerMoore(s, alphabet='ACGT')

In [11]:
boyer_moore(s, s_bm, genome)

([56922], 127974)

In [13]:
import bisect
import sys

class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

def read_genome(filename):
    """Read a genome from a FASTA or text file"""
    genome = ''
    with open(r"C:\Users\liamm\Desktop\chr1.GRCh38.excerpt.fasta", 'r') as f:
        for line in f:
            if not line.startswith('>'):  # skip header lines
                genome += line.rstrip()
    return genome

def reverse_complement(s):
    """Return the reverse complement of a DNA string"""
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    return ''.join(complement.get(base, base) for base in reversed(s))

def approximate_match_index_with_hit_count(p, t, index, n=2, include_reverse=False):
   
    segment_length = index.k  # 8-mer length
    all_matches = set()  # Use a set to avoid counting matches multiple times
    total_hits = 0  # Counter for total index hits
    match_details = {
        'forward': {},  
        'reverse': {}   
    }
    
    patterns = [(p, 'forward')]
    if include_reverse:
        patterns.append((reverse_complement(p), 'reverse'))
    
    for pattern, direction in patterns:
        # Divide the pattern into n+1 segments to guarantee at least one exact match
        for i in range(n + 1):
            start = i * segment_length
            end = min((i + 1) * segment_length, len(pattern))
            
            # Skip if segment would be too short (edge case)
            if end - start < segment_length:
                continue
            
            # Use index to find exact matches for this segment
            segment = pattern[start:end]
            hits = index.query(segment)
            
            # Count total index hits for this segment
            total_hits += len(hits)
            
            # Check each potential match
            for hit in hits:
                # Adjust hit position to account for segment offset
                hit_adjusted = hit - start
                
                # Skip if adjusted position would be out of bounds
                if hit_adjusted < 0 or hit_adjusted + len(pattern) > len(t):
                    continue
                
                # Count mismatches between pattern and text at adjusted position
                mismatches = 0
                for j in range(len(pattern)):
                    if hit_adjusted + j < len(t) and pattern[j] != t[hit_adjusted + j]:
                        mismatches += 1
                        if mismatches > n:
                            break
                
                # Add position to matches if within mismatch threshold
                if mismatches <= n:
                    all_matches.add(hit_adjusted)
                    match_details[direction][hit_adjusted] = mismatches
    
    return sorted(list(all_matches)), total_hits, match_details

def main():
    """Main function to run the approximate matching algorithm"""
    # Alu-derived pattern
    p = "GGCGCGGTGGCTCACGCCTGTAAT"
    
    # Check if a filename is provided as a command-line argument
    if len(sys.argv) > 1:
        filename = sys.argv[1]
    else:
        filename = input("Enter the path to the chromosome file: ")
    
    # Read the chromosome sequence
    print(f"Reading genome from {filename}...")
    t = read_genome(filename)
    print(f"Genome length: {len(t)} bp")
    
    # Build the index
    print("Building 8-mer index...")
    index = Index(t, 8)
    print("Index built successfully")
    
    # Find approximate matches without reverse complements (as specified in the problem)
    print("Finding approximate matches (up to 2 mismatches, forward strand only)...")
    matches, total_hits, match_details = approximate_match_index_with_hit_count(p, t, index, n=2, include_reverse=False)
    
    print(f"\nResults for forward strand only:")
    print(f"The Alu-derived pattern occurs {len(matches)} times with up to 2 mismatches")
    print(f"Total index hits during search: {total_hits}")
    
    # If you want to include reverse complements, uncomment the following:
    """
    print("\nNow searching with reverse complements included...")
    matches_with_rc, total_hits_with_rc, match_details_with_rc = approximate_match_index_with_hit_count(
        p, t, index, n=2, include_reverse=True)
    
    forward_matches = sum(1 for pos in match_details_with_rc['forward'])
    reverse_matches = sum(1 for pos in match_details_with_rc['reverse'])
    
    print(f"\nResults including reverse complements:")
    print(f"The Alu-derived pattern occurs {len(matches_with_rc)} times total with up to 2 mismatches")
    print(f"Forward strand matches: {forward_matches}")
    print(f"Reverse strand matches: {reverse_matches}")
    print(f"Total index hits during search: {total_hits_with_rc}")
    """
    
    # Print the first few matches for verification
    if matches:
        print("\nFirst 5 match positions:", matches[:5])
        print("\nSample matches (position, sequence, mismatches):")
        count = 0
        for pos in sorted(match_details['forward'].keys()):
            if count >= 3:
                break
            match_seq = t[pos:pos+len(p)]
            mismatches = match_details['forward'][pos]
            print(f"Position {pos}: {match_seq}, Mismatches: {mismatches}")
            count += 1

if __name__ == "__main__":
    main()

Reading genome from -f...
Genome length: 800000 bp
Building 8-mer index...
Index built successfully
Finding approximate matches (up to 2 mismatches, forward strand only)...

Results for forward strand only:
The Alu-derived pattern occurs 19 times with up to 2 mismatches
Total index hits during search: 90

First 5 match positions: [56922, 84641, 147558, 160162, 160729]

Sample matches (position, sequence, mismatches):
Position 56922: GGCGCGGTGGCTCACGCCTGTAAT, Mismatches: 0
Position 84641: GGCGCGGTGGCTCATGCCTGTAAT, Mismatches: 1
Position 147558: GGCGCGGTGGCTCATGCCTGTAAT, Mismatches: 1


In [19]:
import bisect

class SubseqIndex(object):
    """ Holds a subsequence index for a text T """

    def __init__(self, t, k, ival):
        """
        Initializes a SubseqIndex object.

        Args:
            t (str): The text (genome) to be indexed.
            k (int): The k-mer length for the index.
            ival (int): The interval between indexed k-mers.
        """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq

    def query(self, p):
        """
        Return index hits for first subseq of p.

        Args:
            p (str): The pattern to query.

        Returns:
            list: A list of indices in t where the k-mer of p occurs.
        """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


def find_approximate_matches_24mer(pattern, text, index, max_mismatches=2):
    """
    Finds approximate matches of a 24-length pattern in a text using a SubseqIndex
    with k=8 and ival=3, allowing up to 2 mismatches.

    Args:
        pattern (str): The pattern to search for (length 24).
        text (str): The text (genome) to search in.
        index (SubseqIndex): A SubseqIndex object built on the text.
        max_mismatches (int, optional): The maximum number of allowed mismatches. Defaults to 2.

    Returns:
        tuple: A tuple containing:
            - list: A list of indices in text where approximate matches of the pattern occur.
            - int: The total number of index hits.
    """
    if len(pattern) != 24:
        raise ValueError("Pattern length must be 24")
    if max_mismatches > 2:
        raise ValueError("Maximum mismatches allowed is 2")

    hits = index.query(pattern)
    total_index_hits = len(hits)  # Count the total number of index hits
    matches = []

    for hit in hits:
        mismatches = 0
        for i in range(len(pattern)):
            if text[hit + i] != pattern[i]:
                mismatches += 1
                if mismatches > max_mismatches:
                    break
        if mismatches <= max_mismatches:
            matches.append(hit)

    return sorted(list(set(matches))), total_index_hits  # Remove duplicates and sort, return total hits



def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

if __name__ == '__main__':
    # Example usage:
    filename = r"C:\Users\liamm\Desktop\chr1.GRCh38.excerpt.fasta"  
    genome = readGenome(filename)
    pattern_24mer = "GGCGCGGTGGCTCACGCCTGTAAT"  
    k = 8
    ival = 3
    index = SubseqIndex(genome, k, ival)
    approximate_matches_24mer, total_hits_24mer = find_approximate_matches_24mer(pattern_24mer, genome, index)
    print(f"Approximate matches (up to 2 mismatches) for 24-mer: {approximate_matches_24mer}")
    print(f"Total number of index hits for 24-mer: {total_hits_24mer}")


Approximate matches (up to 2 mismatches) for 24-mer: [56922, 84641, 147558, 191452, 262042, 273669, 364263, 465647, 635931, 657496, 681737, 717706, 747359]
Total number of index hits for 24-mer: 35
